# Les enumerations

---

 ## Une forme commune a d'autres langages : 
 
 
 - Une énumération en Rust est un type qui représente des données qui sont l'une des nombreuses variantes possibles.. 
 
 `enum` `EnumName` `{ Variant1, Variant2, ...}`

In [2]:
#[derive(Debug, PartialEq)] // Ne faite pas attention pour l'instant a cette ligne !
enum Error {
    FileNotFound,
    BadFormat,
    Unexpected,
}

- Ici Error sera soit assigner au premier variant, soit au second ou soit au troisième. La syntaxe des namespaces avec `::` sera utilisée pour nommer le variant comme si le type de l'enumeration est un namespace.

In [44]:
let e: Error = Error::Unexpected;

dbg!(e);

[src/lib.rs:53] e = Unexpected


`let` `mon_enum_var_name` `:` `EnumName` `=` `EnumName` `::` `VariantName`

ou

`let` `mon_enum_var_name` `=` `EnumName` `::` `VariantName`

*Ne vous souciez pas pour l'instant de ce **derive(Debug)**, il est nécessaire pour la compilation, nous le verrons en détail au chapitre suivant.*

- Comme le type de l'enumeration est un namespace, la syntaxe suivante fonctionne aussi :

In [45]:
// On importe ici les noms des variants dans le scope
use Error::*;

// Plus besoin d'utiliser Error::Unexpected puisque le compilateur sait deja ce qu'est Unexpected.
let e: Error = Unexpected;

dbg!(e);

[src/lib.rs:54] e = Unexpected


- Enfin, il est possible de tester la valeur de Error. Ici le **derive(PartialEq)** est malheureusement nécessaire afin que le code compile, nous verrons ça aussi en détail au chapitre suivant.

In [46]:
let e: Error = Error::BadFormat;

if e == Error::Unexpected {
    println!("Unexpected !");
} else if e == Error::BadFormat {
    println!("Bad Format !");
} else {
    println!("File not found !");
}


Bad Format !


()

- Pour des types numériques, on peut aussi forcer la représentation des valeurs d'une énumération par un type primitif tel u32 via la directive `#[repr(u32)]`, ceci peut avoir son importance si l'on veut interfacer le code Rust avec du C par exemple. cf. FFI

In [49]:
#[derive(Debug, PartialEq)]
#[repr(u32)]
enum Error {
    FileNotFound = 0,
    BadFormat = 7,
    Unexpected = 11,
}

let err = Error::BadFormat;
dbg!(&err);
dbg!(err as u32); // Ca permet le cast en u32 !

let err = Error::BadFormat;
if err as u32 == 7 {
    println!("Bad Format !");
}

[src/lib.rs:55] &err = BadFormat
[src/lib.rs:56] err as u32 = 7


Bad Format !


()

> Il est ainsi possible de caster l'enumeration en u32.

## L'association a des donnees:

 - Contrairement à beaucoup d'autres langages, Rust permet d'associer de véritables set de données aux variants des énumérations:

In [48]:
#[derive(Debug)]
struct Lada {
    model: String,
    max_speed: usize,
    capacity: usize,
}

#[derive(Debug)]
struct Wolkswagen {
    model: String,
    max_speed: usize,
}

#[derive(Debug)]
struct Bike;

#[derive(Debug)]
enum Vehicle {
    Lada(Lada),
    Wolkswagen(Wolkswagen),
    Bike(Bike),
    Unknown,
    Unexpected(usize),
    None,
}

// v sera un vehicule de type Wolkswagen
let mut v = Vehicle::Wolkswagen(Wolkswagen{
    model: "BMW".to_string(),
    max_speed: 180,
});

dbg!(v);

v = Vehicle::None;
dbg!(v);

v = Vehicle::Unexpected(126);
dbg!(v);

[src/lib.rs:59] v = Wolkswagen(
    Wolkswagen {
        max_speed: 180,
    },
        model: "BMW",
)
[src/lib.rs:62] v = None
[src/lib.rs:65] v = Unexpected(
    126,
)


*Bien que ce ne soit en rien une obligation, il est fréquent de rencontrer les mêmes noms pour les variants et les structures contenues.*

## Exercices
### Le type Link

 - En C, une liste chaînée est composée de plusieurs maillons qui contiennent chacun une valeur et un pointeur sur le maillon suivant, le dernier maillon de la liste possède un pointeur nul.
 
```
#include <stddef.h>
#include <stdio.h>

struct Link {
     struct Link *next;
     unsigned int value;
 };

int main(void) {
    struct Link list1, list2;
    list1.next = &list2;
    list1.value = 11;
    list2.next = NULL;
    list2.value = 22;

    printf("l1 val = %i, l2 val = %i next_ptr = %p\n", list1.value, list1.next->value, list1.next->next);
}
```
**sortie du programme:** 

l1 val = 11, l2 val = 22 next_ptr = (nil)

**QUESTION:** 
Si on veut faire une liste chaînée en Rust sans utiliser de pointeur, quel pourrait être la définition du type Link ou d'autres éléments s'ils sont nécessaires ?

L'idée pourrait être d'utiliser une structure qui contient deux champs, un champ valeur et un champ next comme on fait en C. Le champ next sera de type d'une énumération qui aurait comme variant soit une nouvelle structure de type Link, soit un variant que l'on peut nommer None.
```
    struct Link {
        next: Next,
        value: usize,
    }

    enum Next {
        Link(Box<Link>),
        None,
    }
```
*Une autre spécificité du Rust concerne le fait que l'on ne peut avoir des structures de taille variables et non déterminées a la compilation, ainsi, je suis obligé à déclarer une **Box** qui est en fait une allocation dynamique, ainsi la taille de la Box est fixe, c'est celle d'un pointeur en mémoire. Le code ci-dessous est donc faux :
```
    struct Link {
        next: Next,
        value: usize,
    }

    enum Next {
        Link(Link), // FAUX !
        None,
    }
```
**ERROR:** rustc: recursive types `Link` and `Next` have infinite size [E0072]

- Rust implémente déjà dans la std une énumération qui contient un variant None, ce sont les **Options**. Nous les verrons plus tard. avec Option, le code final d'une liste chainee en rust serait ainsi:
```
    struct Link {
        next: Option<Box<Link>>,
        value: usize,
    }
```